In [1]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup
import csv
import os

## 검색 키워드
- 관광지 + 여행 + 후기
- 관광지 + 여행 + 주말에
- 관광지 + 여행 + 대전 
- 관광지 + 관광 + 후기
- 관광지 + 관광 + 주말에 
- 관광지 + 관광 + 대전 

## 관광지 리스트
- 총 12개 + 4개    


- 오-월드
- 뿌리공원
- 엑스포과학공원
- 한밭수목원
- 계족산황톳길
- 대청호반
- 장태산휴양림
- 대전둘레산길
- 동춘당
- 대전문화예술단지 : 대전예술의전당, 대전시립미술관, 이응노미술관
- 으능정이문화의거리
- 유성온천



- 성심당
- 시민천문대
- 수통골
- 한밭야구장
- 국립중앙과학관

In [2]:
place = ['오월드', '뿌리공원','엑스포과학공원','한밭수목원','계족산황톳길','대청호반','장태산휴양림','대전둘레산길','동춘당','대전문화예술단지',
         '으능정이문화의거리','유성온천','성심당','시민천문대','수통골','한밭야구장']
print(len(place))
tour = ['여행', '놀러']
keywords = ['후기', '대전', '주말에']

complete_keyword = [f'{p} {t} {w}' for p in place for t in tour for w in keywords]

print(len(complete_keyword))

16
96


In [3]:
complete_keyword

['오월드 여행 후기',
 '오월드 여행 대전',
 '오월드 여행 주말에',
 '오월드 놀러 후기',
 '오월드 놀러 대전',
 '오월드 놀러 주말에',
 '뿌리공원 여행 후기',
 '뿌리공원 여행 대전',
 '뿌리공원 여행 주말에',
 '뿌리공원 놀러 후기',
 '뿌리공원 놀러 대전',
 '뿌리공원 놀러 주말에',
 '엑스포과학공원 여행 후기',
 '엑스포과학공원 여행 대전',
 '엑스포과학공원 여행 주말에',
 '엑스포과학공원 놀러 후기',
 '엑스포과학공원 놀러 대전',
 '엑스포과학공원 놀러 주말에',
 '한밭수목원 여행 후기',
 '한밭수목원 여행 대전',
 '한밭수목원 여행 주말에',
 '한밭수목원 놀러 후기',
 '한밭수목원 놀러 대전',
 '한밭수목원 놀러 주말에',
 '계족산황톳길 여행 후기',
 '계족산황톳길 여행 대전',
 '계족산황톳길 여행 주말에',
 '계족산황톳길 놀러 후기',
 '계족산황톳길 놀러 대전',
 '계족산황톳길 놀러 주말에',
 '대청호반 여행 후기',
 '대청호반 여행 대전',
 '대청호반 여행 주말에',
 '대청호반 놀러 후기',
 '대청호반 놀러 대전',
 '대청호반 놀러 주말에',
 '장태산휴양림 여행 후기',
 '장태산휴양림 여행 대전',
 '장태산휴양림 여행 주말에',
 '장태산휴양림 놀러 후기',
 '장태산휴양림 놀러 대전',
 '장태산휴양림 놀러 주말에',
 '대전둘레산길 여행 후기',
 '대전둘레산길 여행 대전',
 '대전둘레산길 여행 주말에',
 '대전둘레산길 놀러 후기',
 '대전둘레산길 놀러 대전',
 '대전둘레산길 놀러 주말에',
 '동춘당 여행 후기',
 '동춘당 여행 대전',
 '동춘당 여행 주말에',
 '동춘당 놀러 후기',
 '동춘당 놀러 대전',
 '동춘당 놀러 주말에',
 '대전문화예술단지 여행 후기',
 '대전문화예술단지 여행 대전',
 '대전문화예술단지 여행 주말에',
 '대전문화예술단지 놀러 후기',
 '대전문화예술단지 놀러 대전',
 '대전문화예술단지 놀러 주말에',
 '으능정이문화

In [4]:
new_list = ['대전예술의전당', '대전시립미술관', '이응노미술관', '국립중앙과학관']

tour = ['여행', '놀러']
keywords = ['후기', '대전', '주말에']

new_keyword = [f'{p} {t} {w}' for p in new_list for t in tour for w in keywords]

print(new_keyword)

['대전예술의전당 여행 후기', '대전예술의전당 여행 대전', '대전예술의전당 여행 주말에', '대전예술의전당 놀러 후기', '대전예술의전당 놀러 대전', '대전예술의전당 놀러 주말에', '대전시립미술관 여행 후기', '대전시립미술관 여행 대전', '대전시립미술관 여행 주말에', '대전시립미술관 놀러 후기', '대전시립미술관 놀러 대전', '대전시립미술관 놀러 주말에', '이응노미술관 여행 후기', '이응노미술관 여행 대전', '이응노미술관 여행 주말에', '이응노미술관 놀러 후기', '이응노미술관 놀러 대전', '이응노미술관 놀러 주말에', '국립중앙과학관 여행 후기', '국립중앙과학관 여행 대전', '국립중앙과학관 여행 주말에', '국립중앙과학관 놀러 후기', '국립중앙과학관 놀러 대전', '국립중앙과학관 놀러 주말에']


## 데이터 저장 경로

In [5]:
data_dir = os.getenv('HOME') + '/Projects/crawling_data'
data_dir

'/home/aiffel-dj26/Projects/crawling_data'

## 크롤링 
- naver 검색 API

In [6]:
naver_client_id = "lzPSlFTEnz232oLdqwzy"
naver_client_secret = "1Ig7U84bjP"

In [7]:
def naver_blog_crawling(data_dir, search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count, fromdate=False, todate=False)
    get_blog_post(data_dir, search_blog_keyword, display_count, search_result_blog_page_count, sort_type, fromdate=False, todate=False)


def get_blog_search_result_pagination_count(search_blog_keyword, display_count, fromdate, todate):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
#     url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_keyword}from{fromdate}to{todate}" 
    url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_keyword}" 

    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8')) # 긁어온 내용 

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = response_body_dict['total']
#             blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))
            

            if blog_pagination_total_count >= 1000:
                blog_pagination_count = 500
            else:
                blog_pagination_count = 500

            print("키워드 " + search_blog_keyword + "에 해당하는 포스팅 수 : " + str(response_body_dict['total']))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 실제 페이징 수 : " + str(blog_pagination_total_count))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 처리할 수 있는 페이징 수 : " + str(blog_pagination_count))

        return blog_pagination_count


def get_blog_post(data_dir, search_blog_keyword, display_count, search_result_blog_page_count, sort_type, fromdate, todate):
    crawl_count = 0
    # csv 파일 생성 및 쓰기
    file = open(f'{data_dir}/{search_blog_keyword}_tmp.csv', 'w', encoding='utf-8', newline='') 
    csvWriter = csv.writer(file)
    
    encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)

    for i in range(1, search_result_blog_page_count + 1):
#         url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_blog_keyword}from{fromdate}to{todate}" 
        url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_blog_keyword}" 
        
#         if fromdate and todate :
#             url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
#                 display_count) + "&start=" + str(i) + "&sort=" + sort_type + f"&from{fromdate}to{todate}"
#         else:
#             url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
#                 display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)
        
        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()

        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))

            for j in range(0, len(response_body_dict['items'])):
                try:
                    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")
                    
                    get_blog_post_content_code = requests.get(blog_post_url)
                    get_blog_post_content_text = get_blog_post_content_code.text

                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                    for link in get_blog_post_content_soup.select('iframe#mainFrame'):
                        real_blog_post_url = "http://blog.naver.com" + link.get('src')
                        

                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text

                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
                            blog_post_content_text = blog_post_content.get_text()
                            
                            if blog_post_content_text is None:
                                print('No text !')
                            remove_html_tag = re.compile('<.*?>')

                            blog_post_title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                            blog_post_description = re.sub(remove_html_tag, '',
                                                           response_body_dict['items'][j]['description'])
                            blog_post_postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],
                                                                            "%Y%m%d").strftime("%y.%m.%d")
                            blog_post_blogger_name = response_body_dict['items'][j]['bloggername']
                            blog_post_full_contents = str(blog_post_content_text)
                            
                            
                            print(blog_post_url)
                            print(blog_post_title)
                            print(blog_post_description)
                            print('-----')
#                             csvWriter.writerow([blog_post_postdate, blog_post_url, blog_post_title, blog_post_description, blog_post_full_contents, blog_post_blogger_name])
                            crawl_count +=1
                            
                            if crawl_count >= 500:
                                file.close()
                                print('finish crawling !')
                                print('crawl count :', crawl_count)
                                return

                except:
                    j += 1
            
    file.close()
    print('finish crawling !')
    print('crawl count :', crawl_count)
    return

In [53]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup



def naver_blog_crawling(search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)
    get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type)


def get_blog_search_result_pagination_count(search_blog_keyword, display_count):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_keyword
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))
            # 블로그 글 1000개 넘으면 1000개만 수집하게 됨
            if blog_pagination_total_count >= 1000:
                # 밑에 카운트 개수가 포스트 1000개 넘을때 따올 포스팅 수임
                blog_pagination_count = 1000
            else:
                blog_pagination_count = blog_pagination_total_count

            print("키워드 " + search_blog_keyword + " 에 해당하는 포스팅 수 : " + str(response_body_dict['total']))
            print("키워드 " + search_blog_keyword + " 에 해당하는 블로그 페이지 수 : " + str(blog_pagination_total_count))
            print("키워드 " + search_blog_keyword + " 에 해당하는 블로그 처리할 수 있는 페이지 수 : " + str(blog_pagination_count))

        return blog_pagination_count


def get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type):
    iter1 = 0
    #여기 내 경로로 해놨는데, 너꺼에 맞게 바꿔 써 텍스트 파일 파싱해주는 거임
    file = open("/Users/donsdev/python-workspace/crawling1/crawl_blog.txt","w", encoding='utf-8')
    encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)

    for i in range(1, search_result_blog_page_count + 1):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()

        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))

            for j in range(0, len(response_body_dict['items'])):
                try:
                    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

                    get_blog_post_content_code = requests.get(blog_post_url)
                    get_blog_post_content_text = get_blog_post_content_code.text

                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                    for link in get_blog_post_content_soup.select('iframe#mainFrame'):
                        real_blog_post_url = "http://blog.naver.com" + link.get('src')

                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text

                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
                            blog_post_content_text = blog_post_content.get_text()

                            remove_html_tag = re.compile('<.*?>')

                            blog_post_title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                            blog_post_description = re.sub(remove_html_tag, '',
                                                           response_body_dict['items'][j]['description'])
                            blog_post_postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],
                                                                            "%Y%m%d").strftime("%y.%m.%d")
                            blog_post_blogger_name = response_body_dict['items'][j]['bloggername']
                            blog_post_full_contents = str(blog_post_content_text)
                            iter1 = iter1 + 1
                            file.write("포스팅 URL : " + blog_post_url + '\n')
                            file.write("포스팅 제목 : " + blog_post_title + '\n')
                            file.write("포스팅 설명 : " + blog_post_description + '\n')
                            file.write("포스팅 날짜 : " + blog_post_postdate + '\n')
                            file.write("블로거 이름 : " + blog_post_blogger_name  + '\n')
                            file.write("포스팅 내용 : " + blog_post_full_contents + '\n\n\n\n')
                            print(iter1)

                except:
                    j += 1

    file.close()

if __name__ == '__main__':
    naver_blog_crawling("오월드 여행 후기", 1, "sim")

키워드 오월드 여행 후기 에 해당하는 포스팅 수 : 458
키워드 오월드 여행 후기 에 해당하는 블로그 페이지 수 : 458
키워드 오월드 여행 후기 에 해당하는 블로그 처리할 수 있는 페이지 수 : 458


FileNotFoundError: [Errno 2] No such file or directory: '/Users/donsdev/python-workspace/crawling1/crawl_blog.txt'

In [7]:
def naver_blog_crawling(data_dir, search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)
    get_blog_post(data_dir, search_blog_keyword, display_count, search_result_blog_page_count, sort_type)


def get_blog_search_result_pagination_count(search_blog_keyword, display_count):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
#     url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_keyword}from{fromdate}to{todate}" 
    url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_keyword}" 
    
#     if fromdate and todate:
#         url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_keyword}from{fromdate}to{todate}" 
#     else:
#         url = f"https://openapi.naver.com/v1/search/blog?&query={encode_search_keyword}" 

    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8')) # 긁어온 내용 

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = response_body_dict['total']
#             blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))
            blog_pagination_count = blog_pagination_total_count
            

#             if blog_pagination_total_count >= 1000:
#                 blog_pagination_count = 500
#             else:
#                 blog_pagination_count = blog_pagination_total_count

            print("키워드 " + search_blog_keyword + "에 해당하는 포스팅 수 : " + str(response_body_dict['total']))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 실제 페이징 수 : " + str(blog_pagination_total_count))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 처리할 수 있는 페이징 수 : " + str(blog_pagination_count))

        return blog_pagination_count


def get_blog_post(data_dir, search_blog_keyword, display_count, search_result_blog_page_count, sort_type):
    crawl_count = 0
    # csv 파일 생성 및 쓰기
    file = open(f'{data_dir}/{search_blog_keyword}_tmp.csv', 'w', encoding='utf-8', newline='') 
    csvWriter = csv.writer(file)
    
    encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)

    for i in range(1, search_result_blog_page_count + 1):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
                display_count) + "&start=" + str(i) + "&sort=" + sort_type 
        
#         if fromdate and todate :
#             url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
#                 display_count) + "&start=" + str(i) + "&sort=" + sort_type + f"&from{fromdate}to{todate}"
#         else:
#             url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
#                 display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)
        
        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()

        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))

            for j in range(0, len(response_body_dict['items'])):
                try:
                    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")
                    
                    get_blog_post_content_code = requests.get(blog_post_url)
                    get_blog_post_content_text = get_blog_post_content_code.text

                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                    for link in get_blog_post_content_soup.select('iframe#mainFrame'):
                        real_blog_post_url = "http://blog.naver.com" + link.get('src')
                        

                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text

                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
                            blog_post_content_text = blog_post_content.get_text()
                            
                            if blog_post_content_text is None:
                                print('No text !')
                            remove_html_tag = re.compile('<.*?>')

                            blog_post_title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                            blog_post_description = re.sub(remove_html_tag, '',
                                                           response_body_dict['items'][j]['description'])
                            blog_post_postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],
                                                                            "%Y%m%d").strftime("%y.%m.%d")
                            blog_post_blogger_name = response_body_dict['items'][j]['bloggername']
                            blog_post_full_contents = str(blog_post_content_text)
                            
#                             if blog_post_postdate >= '19.01.01' and blog_post_postdate <= '21.05.01':
#                                 print(blog_post_url)
#                                 print(blog_post_title)
#                                 print(blog_post_description)
#                                 print(blog_post_postdate)
#                                 print('-----')

                            csvWriter.writerow([blog_post_postdate, blog_post_url, blog_post_title, blog_post_description, blog_post_full_contents, blog_post_blogger_name])
                            crawl_count +=1

                            if crawl_count % 50 == 0:
                                print('now count :', crawl_count)
                                print(blog_post_url)
                                print(blog_post_title)
                                print(blog_post_description)
                                print('-----')

#                             if crawl_count == 500:
#                                 file.close()
#                                 print('finish crawling !')
#                                 print('crawl count :', crawl_count)
#                                 return

                except:
                    j += 1
            
    file.close()
    print('finish crawling !')
    print('crawl count :', crawl_count)
    return

In [9]:
fromdate = 20160101
todate = 20210501

In [8]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "sim")     # 20:27

키워드 오월드여행대전에 해당하는 포스팅 수 : 10475
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 10475
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 10475
now count : 50
https://blog.naver.com/ejzzang29?Redirect=Log&logNo=220296850090
대전여행중 아이와 갈만한곳 오월드
대전여행중 아이와 갈만한곳 오월드 대전을 가긴갔다! 근데.. 아는곳이 없으니.. 신랑지인의 소개로.. 가본 오월드~ 오월드는... 현장예매임돠! 줄 엄청 서있고!!! 한 20분 이상을 서서 기다렸다가 표를... 
-----
now count : 100
https://blog.naver.com/unjung24?Redirect=Log&logNo=220017863102
[대전여행]대전오월드~동물구경#2
[대전여행/대전오월드] 대구에서 볼수없었던 여러종의 동물들을 볼수 있었어요. 엄마빼고 단체샷~! 호랑이, 표범, 제규어...등을 보거나서 찍은거랍니다. 채니는 호랑이랑 둘이서만 찍고 싶데서..^^ 늑대 구경중... 
-----
now count : 150
https://blog.naver.com/akme5684?Redirect=Log&logNo=140191472909
대전 오월드 가족여행 / 대전 오월드 입장료 및 할인카드
싶어 큰 맘 먹고 저번 주말에 여행을 다녀 왔답니다 ^^* 나은이가 신나게 놀 수 있고 운전이 피곤하지 않게 대구에서 2시간정도면 다녀 올 수 있는 대전오월드로 ~~고~~고~~ 대전오월드의 입장시간은 09... 
-----
now count : 200
https://blog.naver.com/lovelyr929?Redirect=Log&logNo=220464836622
대전 여행 / 대전 동물원 * 오월드
대전여행의 제일 큰 목적은 '오월드'에 가는것이였다 첨으로 친구랑 둘이 여행간거였는데 정말 알차고 재밌게 놀았던가같다ㅋㅋㅋㅋ 외국인이 찍어준 사진은.. 호랑이 머리없이 

now count : 1800
https://blog.naver.com/nono1114?Redirect=Log&logNo=150143736776
대전여행 : 대전오월드 구)대전동물원
대전 오월드 대전광역시 중구 사정동 산 39-1번지 054)580-4820 홈피 주소 : http://www.oworld.kr/ ZOO LAND, Flower Land, Joy Land 3가지 테마로 구성. 동대구역에서 대전역까지는 KTX를 타고 가면 50분이 채 안걸리는 거리다.... 
-----
now count : 1850
https://blog.naver.com/hyojun0272?Redirect=Log&logNo=220614549155
대전여행 오월드 자유이용권로 놀이기구 즐기기
:: 대전여행 오월드 자유이용권로 놀이기구 즐기기 :: 간만의 대전여행 폿팅이죠?ㅋㅋㅋㅋ 작년 여름 대전여행 갔을때의 기억을 새록새록 떠올리면서 오월드 방문기를 풀어보도록 하겠슙니당 저희가 대구에서... 
-----
now count : 1900
https://blog.naver.com/ilove0502i?Redirect=Log&logNo=220155960934
[대전여행]대전오월드 사파리,대전성심당
&lt; 대전핫플레이스가 유성아이가?아닌가..아님말규.. 대전은 출장만 가봤지..이렇게 오빠야랑 여행이라... 대전 오!월드에 도착합니다!! 사실 사람이 너무...많아서..아니 주차장에 차가 가득가득해서 그냥 갈까..... 
-----
now count : 1950
https://blog.naver.com/na7771?Redirect=Log&logNo=220153408996
[2014/4/4] 대전여행-대전 오월드-대전동물원-허브향기
올 봄...4월에 한 여행을 이제서야..후후.. 아빠가 이직을 하면서 시간이 생겨 드됴 우리세식구도... 대전오월드.. 평일이라 그런지 사람도 많지않고 좋았다. 사파리가 있어서 추가요금내고 입장했는데..... 
-----
now count : 2000
https://bl

now count : 3550
https://blog.naver.com/zippozip?Redirect=Log&logNo=221781483855
대전오월드 자유이용권 야놀자특가로 득템!
남아서 대전 오월드에 갔었거든요. 그 때 단풍이 한창 예쁘게 피어있을 때여서 단풍여행으로 놀러 가도... 비가 올 지도 모르겠다는 생각에 날씨를 검색해 봤는데 내일 모레 날씨는 흐림으로 떠서... 
-----
now count : 3600
https://blog.naver.com/idyye?Redirect=Log&logNo=100184119776
대전여행 &quot;오월드&quot; 테마가 있는 대전동물원
테마가 있는 대전동물원 &quot;오월드&quot; 지난 주말 여러분들은 어딜 다녀오셨나요~ 저는 산수유 꽃구경하러 가고 싶었는데 마눌님의 거센 반대로 아이들을과 함께 놀아주기 하고 대전 동물원 &quot;오월드&quot;에... 
-----
now count : 3650
https://blog.naver.com/skyline622?Redirect=Log&logNo=150177535435
[대전여행] 오월드 국화축제
바닥에 놓여있던 국화꽃 올해 축제도 작년만큼이나 좋을 것 같다. 나는 올해엔 아마 못 갈 것 같은ㅎㅎ 생각보다 오월드가 멀어서 혼자서 나서기엔 무리인 것 같다. 하하 대전에 살고계신다면 한번 가보시길!
-----
now count : 3700
https://blog.naver.com/unjung24?Redirect=Log&logNo=220014425221
[대전여행]현충원찍고 오월드로~~#1
[대전여행/오월드] 현충일 전 가족모두가 할아버지 산소에 가기로 했답니다. 현충원에 계신 할아버지를 보러 온가족 총출동 큰집가족3명, 우리집 4명, 고모네가족 5명 대가족이 이동하느라 스타렉스 렌트를... 
-----
now count : 3750
https://blog.naver.com/j7447?Redirect=Log&logNo=220781580464
(대전 여행) 오월드

ConnectionResetError: [Errno 104] Connection reset by peer

In [11]:
import pandas as pd

In [26]:
print(len(sample))
drop_sample = sample.drop_duplicates()
print(len(drop_sample))

4135
81


In [24]:
sample = pd.read_csv(data_dir+'/오월드여행대전_tmp.csv', names=['date', 'url', 'title', 'desc', 'text', 'name'])
# print(sample.head())
print(len(sample))
print(len(set(sample.title)))
print(len(set(sample.name)))
print(len(set(sample.url)))
print(set(sample.title))
print(len(sample[sample.date >= '16.01.01']))  # 867
print(len(sample[sample.date >= '19.01.01']))  # 47

4135
81
72
81
{'[대전 여행]가족과 함께 가볼만한곳! 대전 오월드 야간개장', '[충북여행,대전]오월드 버드랜드 너무 재밌어요.', '대전여행 2  ::  대전 동물원/놀이공원 오월드 나들이', '태연이네 가을 여행 - 11월 대전 오월드의 놀이동산', '[대전여행] 친구와 또다시 찾은 대전오월드', '대전여행/대전오월드/으능정이 문화의거리/중앙시장', '생각없이 떠나 온 여행 - 대전 성심당, 오월드', '대전 여행 / 대전 동물원 * 오월드', '[대전여행]대덕특구 게스트하우스 / 대전오월드 사파리 / 대전... ', '대전   오 월드  어린이날  신나는   여행', '외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... ', '[대전여행/오월드 동물원] 대전 아이들과 가볼만한곳, 봄나들이... ', '[대전여행]대전오월드 사파리,대전성심당', '[#20.국내여행] 대전여행 : 대전의 에버랜드 대전오월드... ', '대전여행] 동물원, 꽃세상, 놀이공원이 함께있는 대전오월드', '대전 오월드 출사 여행', '결혼기념일여행, 대전오월드', '[김여사의 체험여행]오~~~~~월드-대전오월드.동물원.사파리.... ', '대전오월드로 가족여행 다녀왔어요!!!/대전오월드/대전동물원... ', '대전여행중 아이와 갈만한곳 오월드', '대전여행 : 대전오월드 구)대전동물원', '[대전가족여행1탄]울산에서 오월드로 동물원에 가는 조각남', '［대전동물원/대전 오월드/대전동물원주변맛집］주말당일여행... ', '대전여행,대전 가볼만한곳-대전오월드 개장시간과 요금', '[2014/4/4] 대전여행-대전 오월드-대전동물원-허브향기', '당일치기 대전 여행 * 대전역, 중앙시장, 오월드, 성심당', '[충남여행/대전여행] 대전오월드/대전동물원 - 14.10.26', '대전 동물원 오월드 입장권 가격 명절나드리', '대전 여행/ 오월드 동물원/ 성심당', '대전 오월드 가족여행 / 대전 오월드 입장료 및 할인카드', '[대전여행]현충원찍고 오월드로~~#1', '

In [23]:
date5y = sample[sample.date >= '16.01.01']
len(set(date5y.url))

18

In [ ]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "sim")    

키워드 오월드여행대전에 해당하는 포스팅 수 : 10475
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 10475
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 10475
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
20.06.16
-----
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
20.06.16
-----
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
20.06.16
-----
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동

In [14]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "sim")    

키워드 오월드여행대전에 해당하는 포스팅 수 : 10474
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 10474
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 500
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
-----
https://blog.naver.com/coldly85?Redirect=Log&logNo=221263275741
대전동물원,대전가볼만한곳,대전여행,대전오월드
대전오월드,대전명소,대전 데이트코스 안냥하세요 ~^^ 자미한 왁싱 고은쌤입니다. 꽃이 완연한 봄이왔어요♥ 꽃 보면서 봄향기 맡으면 마음이 들뜨는 기분이 들어요^^ 봄엔 늘 생각나는 . 동물들도 동면에서... 
-----
https://blog.naver.com/twinkia?Redirect=Log&logNo=110189047719
[대전여행] 대전 오월드 튤립축제, 플라워랜드
[대전여행] 대전 오월드 플라워랜드튤립축제 저는 어릴 적 튤립을 처음보고 “세상에 이렇게 예쁜 꽃이 다 있나”하고 흥분했던 적이 있습니다. 천국에 가본 적은 없지만 아마도 천국에 꽃밭이 있다면 그... 
-----
https://blog.naver.com/na7771?Redirect=Log&logNo=220153408996
[2014/4/4] 대전여행-대전 오월드-대전동물원-허브향기
올 봄...4월에 한 여행을 이제서야..후후.. 아빠가 이직을 하면서 시간이 생겨 드됴 우리세식구도... 대전오월드.. 평일이라 그런지 사람도 많지않고 좋았다. 사파리가 있어서 추가요금내고 입장했는데..... 
-----
https://blog.naver.com/ne

https://blog.naver.com/nono1114?Redirect=Log&logNo=150143736776
대전여행 : 대전오월드 구)대전동물원
대전 오월드 대전광역시 중구 사정동 산 39-1번지 054)580-4820 홈피 주소 : http://www.oworld.kr/ ZOO LAND, Flower Land, Joy Land 3가지 테마로 구성. 동대구역에서 대전역까지는 KTX를 타고 가면 50분이 채 안걸리는 거리다.... 
-----
https://blog.naver.com/j7447?Redirect=Log&logNo=220781580464
(대전 여행) 오월드 야간개장 르미에르 축제에 다녀왔습니다.
모처럼 시간을 맞추게 되서 하루 여행을 떠났어요. 어디 멀리가기엔 어렵고해서... 가까운 대전 오월드 야간 개장을 갔습니다. 오월드에 몇번 와보긴 했지만... 야간개장에 오는건 처음이예요! 들어가는 입구에서... 
-----
https://blog.naver.com/l9890ll9?Redirect=Log&logNo=220552249118
[충남여행/대전여행] 대전오월드/대전동물원 - 14.10.26
대전동물원 14.10.26 날씨짱좋았떤 10월의 어느날 주차하는데만 30분...걸림 ㅋㅋㅋㅋㅋ 뺑글뺑글돌다가 힘겹게 주차하고 오월드 입장!!! 코멘없이 동물사진쭉!! 동물좋아하는 오빠가 열심히 찍은 동물들 ㅎㅎ... 
-----
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
-----
https://blog.naver.com/coldly85?Redirect=Log&logNo=221263275741
대전동물원,대전가볼만한곳,대

https://blog.naver.com/meteorslove?Redirect=Log&logNo=100164800188
[여름휴가/가족여행/대전오월드] 대전유성리베라호텔, 대전오월드
더군다나 오늘은 완전 햇볕 노출인 &lt;대전오월드&gt; 어제 물놀이 탓에 많이 탔는데 오늘도 많이 타겠당!... 가족 올 여름 휴가 2박 3일의 일정을 마무리 하네요. 무엇보다도 엽이가 너무 좋아하고 행복했던 여행이고... 
-----
https://blog.naver.com/wjdwjd9096?Redirect=Log&logNo=130150695996
[대전여행] 대전오월드- 조이랜드, 주랜드
예전 대전동물원이라고 불리던 &lt;대전오월드&gt;를 다녀왔어요. 몇년전 갔을때는 이렇게 잘 꾸며 놓지 않았는데, 플라워랜드가 정말 장관이었어요. 원래 대전동물원으로 출발했던 대전오월드는 지금 놀이공원인... 
-----
https://blog.naver.com/nono1114?Redirect=Log&logNo=150143736776
대전여행 : 대전오월드 구)대전동물원
대전 오월드 대전광역시 중구 사정동 산 39-1번지 054)580-4820 홈피 주소 : http://www.oworld.kr/ ZOO LAND, Flower Land, Joy Land 3가지 테마로 구성. 동대구역에서 대전역까지는 KTX를 타고 가면 50분이 채 안걸리는 거리다.... 
-----
https://blog.naver.com/j7447?Redirect=Log&logNo=220781580464
(대전 여행) 오월드 야간개장 르미에르 축제에 다녀왔습니다.
모처럼 시간을 맞추게 되서 하루 여행을 떠났어요. 어디 멀리가기엔 어렵고해서... 가까운 대전 오월드 야간 개장을 갔습니다. 오월드에 몇번 와보긴 했지만... 야간개장에 오는건 처음이예요! 들어가는 입구에서... 
-----
https://blog.naver.com/l9890ll9?Redirect=Log&logNo=220552249118
[충남여행/대전여행]

https://blog.naver.com/lovelyr929?Redirect=Log&logNo=220464836622
대전 여행 / 대전 동물원 * 오월드
대전여행의 제일 큰 목적은 '오월드'에 가는것이였다 첨으로 친구랑 둘이 여행간거였는데 정말 알차고 재밌게 놀았던가같다ㅋㅋㅋㅋ 외국인이 찍어준 사진은.. 호랑이 머리없이 우리만 덩그러니^^; 그래도... 
-----
https://blog.naver.com/sso5573?Redirect=Log&logNo=220609601481
대전여행  -오~월드 겨울은 정^^말  추워요
하늘과 별의 겨울방학 마지막 여행~~ 2월1일부터 개학하는 겨울방학을 앞두고 대전 오~월드에 왔더래요. 경남에서 대전까지 갈려고 하니 아침 일찍부터 서둘렀는 데~~ 중간점 휴게소에 잠깐 쉬는데 눈이... 
-----
https://blog.naver.com/yotnwjd?Redirect=Log&logNo=140187590477
[대전여행] 오월드 동물원/플라워랜드
대전하면 엑스포가 떠오르는데 거긴.. 초딩 수학여행으로 갔다왔으니 패스 ㅋㅋ 다른곳이 없을까해서 찾아보니 동물원/수목원 정도? 그래서 우린 '오월드' 너로 정했다. 대흥동에서 칼국수를 맛나게 먹고... 
-----
https://blog.naver.com/3dutk?Redirect=Log&logNo=20187361950
[김여사의 체험여행]오~~~~~월드-대전오월드.동물원.사파리.... 
주저마시고 ~대전의 오월드로 가세요~ http://www.oworld.kr/ 애버랜드.롯데월드와 비교하자면 너무나... 녹용이라도먹고 달리기 1등해보렵니다 ㅎㅎ 이번주말. 아이들과 갈때 없으시다구요? 떠나세요~오~월드로!!!
-----
https://blog.naver.com/meteorslove?Redirect=Log&logNo=100164800188
[여름휴가/가족여행/대전오월드] 대전유성리베라호텔, 대전오월드
더군다나 오늘은 완전 햇볕 노출인 &lt;대전오월드&gt; 어제 물놀이 탓에 많이 

https://blog.naver.com/lovelyr929?Redirect=Log&logNo=220496538617
당일치기 대전 여행 * 대전역, 중앙시장, 오월드, 성심당
이번 여행은 오월드가 주목적이였다 오월드 3번째인 나는 그 안에 음식점이 별로라는걸 알고 배를 채우고 가자고 추천했다 대전역앞에 위치한 시장 중앙시장 외관을 보고 작을거라고 생각했는데 엄청나게... 
-----
https://blog.naver.com/choke90?Redirect=Log&logNo=220328512373
[2015.04.04~04.05] 대전여행/대전오월드/대전복합터미널... 
일단 대전하면 제일 유명한 곳이 대전동물원인 오월드!! 몽몽이의 추천이기도 하규! 맘같아선 식목원도... 이번 여행은 미지의 세계 대전에서 나름 선빵하고 온 듯 ㅎㅎㅎ 사실 계획도 제대로 안짜고 그냥 즉흥?... 
-----
https://blog.naver.com/skyline622?Redirect=Log&logNo=150156948183
[대전여행] 오월드, 대전동물원
대전 오월드, 동물원에 가다 어느 하늘이 맑았던 날, 대전 오월드에 나들이를 다녀왔다. 대전 입성... 놀이기구도 있었지만 우리는 놀이기구를 타러 온 것이 아니였기에..패스! △펭귄 이날 꽤 더운편이었는데... 
-----
https://blog.naver.com/angeljin80?Redirect=Log&logNo=220689662575
즐거웠던 대전여행! 대전 오월드 튤립축제,대전갈비집,성심당 :D
즐거웠던 대전여행! 대전 오월드 튤립축제,대전갈비집,성심당 지난 남편님 쉬는 날에..대전을... 성심당 생각이 나서 온 김에 가보자며..얘기 했는데 알고보니 갈비집에서 걸어 갈 수 있는 거리에 있더라구요... 
-----
https://blog.naver.com/lovelyr929?Redirect=Log&logNo=220464836622
대전 여행 / 대전 동물원 * 오월드
대전여행의 제일 큰 목적은 '오월드'에 가는것이였다 첨

...

https://blog.naver.com/jejujcnc?Redirect=Log&logNo=221283126366
대전여행 가볼만한곳 대전 오월드 장미축제
대전여행 가볼만한곳 대전오월드 장미축제에서 재미있게 놀다왔어요. 기간내에 언제라도 가보셔도 좋을꺼에요. 날씨 좋은날 택하시면 더욱 좋답니다. 오늘은 여기까지~
-----
https://blog.naver.com/hwan1279?Redirect=Log&logNo=220647184766
대전오월드/버스여행/대전여행/150725~26
음악을 들어줘야 여행의 맛이쥬. 일을 다 보고 나니, 저녁이 됐고.. 타유를 찾아서 대전으로 내려왔다.... 아주 맛있게 먹었고, 오월드 주차장에 타유를 세우고 꿀밤. 오월드에서 먹는 담날 아침 겸 점심. 맛....없.... 
-----
https://blog.naver.com/hotelist?Redirect=Log&logNo=20166397275
대전여행 제주피플 가족과 대전오월드
대전오월드의 마지막 묘미는 역시나 플라워랜드입니다. 가볍게 산책을 즐기면서 쉴 수 있는 공간인 이곳에서 대전여행의 마침표를 찍고 올 수 있었습니다. ▲ 다른 사람들도 부러워할만한 보너스샷?? 형수님이... 
-----
https://blog.naver.com/ejzzang29?Redirect=Log&logNo=220296850090
대전여행중 아이와 갈만한곳 오월드
대전여행중 아이와 갈만한곳 오월드 대전을 가긴갔다! 근데.. 아는곳이 없으니.. 신랑지인의 소개로.. 가본 오월드~ 오월드는... 현장예매임돠! 줄 엄청 서있고!!! 한 20분 이상을 서서 기다렸다가 표를... 
-----
https://blog.naver.com/minju1004b?Redirect=Log&logNo=90159128779
[대전 여행] 오월드 동물원 &amp; 국화
대전여행 추천지 &quot;오월드 동물원&quot; 대전여행에서 찾은 대전 오월드예요. 놀이동산과 체험전시관뿐 아니라 오월드 동물원에 꽃동산이 함께 있어서 가족, 

https://blog.naver.com/eyecontact02?Redirect=Log&logNo=50174959524
[대전여행]대덕특구 게스트하우스 / 대전오월드 사파리 / 대전... 
[대전여행]대덕특구 게스트하우스 / 대전오월드 사파리 / 대전 사리원면옥 / 대전 성심당 5월마지막주 주말여행 이제서야 사진을 올리게 되느군... 흠.. ㅠ 대구에서 대전으로 고고 ~ 우선 마트부터... 
-----
https://blog.naver.com/leo1227?Redirect=Log&logNo=220054438029
[대전 여행] 오월드 대전 동물원
아침일찍~~ 대전 동물원(오월드)로 달려갔네요~~ 대전 동물원은 쾌적한 시설과 더불어 관리감독이 잘 이루어져 있어 동물들의 컨디션이 매우 좋은 곳입니다. 비교하기 뭐하지만 광주동물원의 경우 청결도도... 
-----
https://blog.naver.com/coldly85?Redirect=Log&logNo=221263275741
대전동물원,대전가볼만한곳,대전여행,대전오월드
대전오월드,대전명소,대전 데이트코스 안냥하세요 ~^^ 자미한 왁싱 고은쌤입니다. 꽃이 완연한 봄이왔어요♥ 꽃 보면서 봄향기 맡으면 마음이 들뜨는 기분이 들어요^^ 봄엔 늘 생각나는 . 동물들도 동면에서... 
-----
https://blog.naver.com/twinkia?Redirect=Log&logNo=110189047719
[대전여행] 대전 오월드 튤립축제, 플라워랜드
[대전여행] 대전 오월드 플라워랜드튤립축제 저는 어릴 적 튤립을 처음보고 “세상에 이렇게 예쁜 꽃이 다 있나”하고 흥분했던 적이 있습니다. 천국에 가본 적은 없지만 아마도 천국에 꽃밭이 있다면 그... 
-----
https://blog.naver.com/na7771?Redirect=Log&logNo=220153408996
[2014/4/4] 대전여행-대전 오월드-대전동물원-허브향기
올 봄...4월에 한 여행을 이제서야..후후.. 아빠가 이직을 하면서 시간이 생겨 드됴 우리세식

https://blog.naver.com/wjdwjd9096?Redirect=Log&logNo=130150695996
[대전여행] 대전오월드- 조이랜드, 주랜드
예전 대전동물원이라고 불리던 &lt;대전오월드&gt;를 다녀왔어요. 몇년전 갔을때는 이렇게 잘 꾸며 놓지 않았는데, 플라워랜드가 정말 장관이었어요. 원래 대전동물원으로 출발했던 대전오월드는 지금 놀이공원인... 
-----
https://blog.naver.com/nono1114?Redirect=Log&logNo=150143736776
대전여행 : 대전오월드 구)대전동물원
대전 오월드 대전광역시 중구 사정동 산 39-1번지 054)580-4820 홈피 주소 : http://www.oworld.kr/ ZOO LAND, Flower Land, Joy Land 3가지 테마로 구성. 동대구역에서 대전역까지는 KTX를 타고 가면 50분이 채 안걸리는 거리다.... 
-----
https://blog.naver.com/j7447?Redirect=Log&logNo=220781580464
(대전 여행) 오월드 야간개장 르미에르 축제에 다녀왔습니다.
모처럼 시간을 맞추게 되서 하루 여행을 떠났어요. 어디 멀리가기엔 어렵고해서... 가까운 대전 오월드 야간 개장을 갔습니다. 오월드에 몇번 와보긴 했지만... 야간개장에 오는건 처음이예요! 들어가는 입구에서... 
-----
https://blog.naver.com/l9890ll9?Redirect=Log&logNo=220552249118
[충남여행/대전여행] 대전오월드/대전동물원 - 14.10.26
대전동물원 14.10.26 날씨짱좋았떤 10월의 어느날 주차하는데만 30분...걸림 ㅋㅋㅋㅋㅋ 뺑글뺑글돌다가 힘겹게 주차하고 오월드 입장!!! 코멘없이 동물사진쭉!! 동물좋아하는 오빠가 열심히 찍은 동물들 ㅎㅎ... 
-----
https://blog.naver.com/killshot1?Redirect=Log&logNo=80196436064
여행기 - 2박3일 기차여행 #7 [

https://blog.naver.com/angeljin80?Redirect=Log&logNo=220689662575
즐거웠던 대전여행! 대전 오월드 튤립축제,대전갈비집,성심당 :D
즐거웠던 대전여행! 대전 오월드 튤립축제,대전갈비집,성심당 지난 남편님 쉬는 날에..대전을... 성심당 생각이 나서 온 김에 가보자며..얘기 했는데 알고보니 갈비집에서 걸어 갈 수 있는 거리에 있더라구요... 
-----
https://blog.naver.com/lovelyr929?Redirect=Log&logNo=220464836622
대전 여행 / 대전 동물원 * 오월드
대전여행의 제일 큰 목적은 '오월드'에 가는것이였다 첨으로 친구랑 둘이 여행간거였는데 정말 알차고 재밌게 놀았던가같다ㅋㅋㅋㅋ 외국인이 찍어준 사진은.. 호랑이 머리없이 우리만 덩그러니^^; 그래도... 
-----
https://blog.naver.com/sso5573?Redirect=Log&logNo=220609601481
대전여행  -오~월드 겨울은 정^^말  추워요
하늘과 별의 겨울방학 마지막 여행~~ 2월1일부터 개학하는 겨울방학을 앞두고 대전 오~월드에 왔더래요. 경남에서 대전까지 갈려고 하니 아침 일찍부터 서둘렀는 데~~ 중간점 휴게소에 잠깐 쉬는데 눈이... 
-----
https://blog.naver.com/yotnwjd?Redirect=Log&logNo=140187590477
[대전여행] 오월드 동물원/플라워랜드
대전하면 엑스포가 떠오르는데 거긴.. 초딩 수학여행으로 갔다왔으니 패스 ㅋㅋ 다른곳이 없을까해서 찾아보니 동물원/수목원 정도? 그래서 우린 '오월드' 너로 정했다. 대흥동에서 칼국수를 맛나게 먹고... 
-----
https://blog.naver.com/3dutk?Redirect=Log&logNo=20187361950
[김여사의 체험여행]오~~~~~월드-대전오월드.동물원.사파리.... 
주저마시고 ~대전의 오월드로 가세요~ http://www.oworld.kr/ 애버랜드.롯데월드와 

https://blog.naver.com/akme5684?Redirect=Log&logNo=140191472909
대전 오월드 가족여행 / 대전 오월드 입장료 및 할인카드
싶어 큰 맘 먹고 저번 주말에 여행을 다녀 왔답니다 ^^* 나은이가 신나게 놀 수 있고 운전이 피곤하지 않게 대구에서 2시간정도면 다녀 올 수 있는 대전오월드로 ~~고~~고~~ 대전오월드의 입장시간은 09... 
-----
https://blog.naver.com/leeh201203?Redirect=Log&logNo=60210855428
[대전여행] 대전 오월드
오월드 갈때 오빠꺼 카드로 긁었어요^^ 할인된다고 해서ㅋㅋㅋ 혹시해서 할인카드 올려드릴께요 ● 대전오월드 주소 : 대전광역시 중구 사정동 산39-1 ☎ 042-580-4820 운영시간 : 09:30~18:00 홈페이지 : www.oworld.kr
-----
https://blog.naver.com/kingofnsb?Redirect=Log&logNo=220154019082
(수학여행 1일차-대전오월드, 대전국립과학관, 시민천문대)
□ 대전오월드 광주패밀리랜드보다는 훨씬 규모가 있는 대전오월드에 갔다. 완전과학문화체험으로 수학여행을 가고 싶었지만 6학년 아이들의 심리에 어느 정도 만족감을 주기 위해서 대전오월드를 포함했다.... 
-----
https://blog.naver.com/hyojun0272?Redirect=Log&logNo=220614549155
대전여행 오월드 자유이용권로 놀이기구 즐기기
:: 대전여행 오월드 자유이용권로 놀이기구 즐기기 :: 간만의 대전여행 폿팅이죠?ㅋㅋㅋㅋ 작년 여름 대전여행 갔을때의 기억을 새록새록 떠올리면서 오월드 방문기를 풀어보도록 하겠슙니당 저희가 대구에서... 
-----
finish crawling !
crawl count : 500


In [ ]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "sim", fromdate, todate)    

키워드 오월드여행대전에 해당하는 포스팅 수 : 10473
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 10473
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 500


In [ ]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "date")    

키워드 오월드여행대전에 해당하는 포스팅 수 : 10471
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 10471
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 1000
https://blog.naver.com/rororo001?Redirect=Log&logNo=222376743475
나의 첫째아들 6번째생일 날씨도 좋고 마음도 좋고 이 기분으로 우리는 여행을 간다 대전오월드로 놀이기구도 태워주고싶고 동물도 보여주고싶고 4번째 방문... 애들이 좋아한다 이번엔 처음으로 놀이기구도... 
-----
https://blog.naver.com/tryxtry?Redirect=Log&logNo=222374169136
1976년 1월 21일 비로소 상업운행을 개시함으로써 콩코드기는 민간 항공여행의 초음속 시대를 열었다.... 육상잡지로는 대한육상경기연맹의 ‘육상월드’(월간)에 이어 두번째다. ‘달리는 사람들’은... 
-----
https://blog.naver.com/csm1499?Redirect=Log&logNo=222373383480
김석희옮김 어린이소설) 1-12 박스세트 4만5천원쿠키들의신나는세계여행쿠키런어드벤처... 큰책) 1-3 &quot;스포츠월드 연재골프만화&quot; 1만원이원복교수의와인의세계세계의와인(이원복글그림... 
-----
https://blog.naver.com/likechoi?Redirect=Log&logNo=222369824388
환생여행1~2완 상급1만9천원 -이원호이원호 황금제국1~4완 상급4만3천원 -이원호이원호 흑야1~2완... 상급7천원 -사이토아카리강원도걷기여행 중급1만5천원 -길을찾는사람들개똥벌레헛간을태우다 1만8천원... 
-----
https://blog.naver.com/goatman0736?Redirect=Log&logNo=222369028018
아버지는 결혼기념일로 딸은 생일기념으로 함께 여행을 떠나게 되는데, 제목 : 묘 장르... 2017 &lt;롯데월드 좀비 팩토리&gt; 하이

https://blog.naver.com/skkim2202?Redirect=Log&logNo=222359071781
조작이 어려운건 아닌데.. 맘대로 안 되는 이유는 뭘까요..~~ #5월의오월들 #햇살좋은날 #나들이 #사람엄청많았음 #봄을느낀 #근교여행 #대전여행 #대전나들이 #오월드
-----
https://blog.naver.com/ghuangzhu1029?Redirect=Log&logNo=222355155377
KBS 월드 K-Rush2 # 11월 27일 Arirang TV Pops in Seoul # 12월 4일 MBN 히어로 콘서트 VCR 12월 6일 MBC... KBS 월드 K-Rush3 #1 #2 4월 29일 ~ 5월 6일 SBS 런닝맨 5월 2일 MBC 어린이에게 새 생명을 VCR # 5월 3일... 
-----
https://blog.naver.com/rororo001?Redirect=Log&logNo=222376743475
나의 첫째아들 6번째생일 날씨도 좋고 마음도 좋고 이 기분으로 우리는 여행을 간다 대전오월드로 놀이기구도 태워주고싶고 동물도 보여주고싶고 4번째 방문... 애들이 좋아한다 이번엔 처음으로 놀이기구도... 
-----
https://blog.naver.com/tryxtry?Redirect=Log&logNo=222374169136
1976년 1월 21일 비로소 상업운행을 개시함으로써 콩코드기는 민간 항공여행의 초음속 시대를 열었다.... 육상잡지로는 대한육상경기연맹의 ‘육상월드’(월간)에 이어 두번째다. ‘달리는 사람들’은... 
-----
https://blog.naver.com/csm1499?Redirect=Log&logNo=222373383480
김석희옮김 어린이소설) 1-12 박스세트 4만5천원쿠키들의신나는세계여행쿠키런어드벤처... 큰책) 1-3 &quot;스포츠월드 연재골프만화&quot; 1만원이원복교수의와인의세계세계의와인(이원복글그림... 
-----
https://blog.naver.com/likechoi?Redi

https://blog.naver.com/skkim2202?Redirect=Log&logNo=222359071781
조작이 어려운건 아닌데.. 맘대로 안 되는 이유는 뭘까요..~~ #5월의오월들 #햇살좋은날 #나들이 #사람엄청많았음 #봄을느낀 #근교여행 #대전여행 #대전나들이 #오월드
-----
https://blog.naver.com/ghuangzhu1029?Redirect=Log&logNo=222355155377
KBS 월드 K-Rush2 # 11월 27일 Arirang TV Pops in Seoul # 12월 4일 MBN 히어로 콘서트 VCR 12월 6일 MBC... KBS 월드 K-Rush3 #1 #2 4월 29일 ~ 5월 6일 SBS 런닝맨 5월 2일 MBC 어린이에게 새 생명을 VCR # 5월 3일... 
-----
https://blog.naver.com/tryxtry?Redirect=Log&logNo=222374169136
1976년 1월 21일 비로소 상업운행을 개시함으로써 콩코드기는 민간 항공여행의 초음속 시대를 열었다.... 육상잡지로는 대한육상경기연맹의 ‘육상월드’(월간)에 이어 두번째다. ‘달리는 사람들’은... 
-----
https://blog.naver.com/csm1499?Redirect=Log&logNo=222373383480
김석희옮김 어린이소설) 1-12 박스세트 4만5천원쿠키들의신나는세계여행쿠키런어드벤처... 큰책) 1-3 &quot;스포츠월드 연재골프만화&quot; 1만원이원복교수의와인의세계세계의와인(이원복글그림... 
-----
https://blog.naver.com/likechoi?Redirect=Log&logNo=222369824388
환생여행1~2완 상급1만9천원 -이원호이원호 황금제국1~4완 상급4만3천원 -이원호이원호 흑야1~2완... 상급7천원 -사이토아카리강원도걷기여행 중급1만5천원 -길을찾는사람들개똥벌레헛간을태우다 1만8천원... 
-----
https://blog.naver.com/goatman0736

In [15]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "sim")    

키워드 오월드여행대전에 해당하는 포스팅 수 : 10471
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 10471
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 1000
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
-----
https://blog.naver.com/coldly85?Redirect=Log&logNo=221263275741
대전동물원,대전가볼만한곳,대전여행,대전오월드
대전오월드,대전명소,대전 데이트코스 안냥하세요 ~^^ 자미한 왁싱 고은쌤입니다. 꽃이 완연한 봄이왔어요♥ 꽃 보면서 봄향기 맡으면 마음이 들뜨는 기분이 들어요^^ 봄엔 늘 생각나는 . 동물들도 동면에서... 
-----
https://blog.naver.com/twinkia?Redirect=Log&logNo=110189047719
[대전여행] 대전 오월드 튤립축제, 플라워랜드
[대전여행] 대전 오월드 플라워랜드튤립축제 저는 어릴 적 튤립을 처음보고 “세상에 이렇게 예쁜 꽃이 다 있나”하고 흥분했던 적이 있습니다. 천국에 가본 적은 없지만 아마도 천국에 꽃밭이 있다면 그... 
-----
https://blog.naver.com/na7771?Redirect=Log&logNo=220153408996
[2014/4/4] 대전여행-대전 오월드-대전동물원-허브향기
올 봄...4월에 한 여행을 이제서야..후후.. 아빠가 이직을 하면서 시간이 생겨 드됴 우리세식구도... 대전오월드.. 평일이라 그런지 사람도 많지않고 좋았다. 사파리가 있어서 추가요금내고 입장했는데..... 
-----
https://blog.naver.com/n

https://blog.naver.com/nono1114?Redirect=Log&logNo=150143736776
대전여행 : 대전오월드 구)대전동물원
대전 오월드 대전광역시 중구 사정동 산 39-1번지 054)580-4820 홈피 주소 : http://www.oworld.kr/ ZOO LAND, Flower Land, Joy Land 3가지 테마로 구성. 동대구역에서 대전역까지는 KTX를 타고 가면 50분이 채 안걸리는 거리다.... 
-----
https://blog.naver.com/j7447?Redirect=Log&logNo=220781580464
(대전 여행) 오월드 야간개장 르미에르 축제에 다녀왔습니다.
모처럼 시간을 맞추게 되서 하루 여행을 떠났어요. 어디 멀리가기엔 어렵고해서... 가까운 대전 오월드 야간 개장을 갔습니다. 오월드에 몇번 와보긴 했지만... 야간개장에 오는건 처음이예요! 들어가는 입구에서... 
-----
https://blog.naver.com/l9890ll9?Redirect=Log&logNo=220552249118
[충남여행/대전여행] 대전오월드/대전동물원 - 14.10.26
대전동물원 14.10.26 날씨짱좋았떤 10월의 어느날 주차하는데만 30분...걸림 ㅋㅋㅋㅋㅋ 뺑글뺑글돌다가 힘겹게 주차하고 오월드 입장!!! 코멘없이 동물사진쭉!! 동물좋아하는 오빠가 열심히 찍은 동물들 ㅎㅎ... 
-----
https://blog.naver.com/freest80?Redirect=Log&logNo=222002619303
외국인과 떠나는 대전여행 오월드 동물원을 찾아서 어서와... 
대전여행하면 빼놓을수 없는곳! TV동물농장에 주로 나왔던 동물원이 대전에 있다 바로 오월드!ㅋ 대전 오월드 동물원 구경 입성 ㅋ 한참 꽃피는 시즌에가면 아주 좋다!ㅋ 온통 꽃동산 장미 이벤트가 한창... 
-----
https://blog.naver.com/coldly85?Redirect=Log&logNo=221263275741
대전동물원,대전가볼만한곳,대

https://blog.naver.com/meteorslove?Redirect=Log&logNo=100164800188
[여름휴가/가족여행/대전오월드] 대전유성리베라호텔, 대전오월드
더군다나 오늘은 완전 햇볕 노출인 &lt;대전오월드&gt; 어제 물놀이 탓에 많이 탔는데 오늘도 많이 타겠당!... 가족 올 여름 휴가 2박 3일의 일정을 마무리 하네요. 무엇보다도 엽이가 너무 좋아하고 행복했던 여행이고... 
-----
https://blog.naver.com/wjdwjd9096?Redirect=Log&logNo=130150695996
[대전여행] 대전오월드- 조이랜드, 주랜드
예전 대전동물원이라고 불리던 &lt;대전오월드&gt;를 다녀왔어요. 몇년전 갔을때는 이렇게 잘 꾸며 놓지 않았는데, 플라워랜드가 정말 장관이었어요. 원래 대전동물원으로 출발했던 대전오월드는 지금 놀이공원인... 
-----
https://blog.naver.com/nono1114?Redirect=Log&logNo=150143736776
대전여행 : 대전오월드 구)대전동물원
대전 오월드 대전광역시 중구 사정동 산 39-1번지 054)580-4820 홈피 주소 : http://www.oworld.kr/ ZOO LAND, Flower Land, Joy Land 3가지 테마로 구성. 동대구역에서 대전역까지는 KTX를 타고 가면 50분이 채 안걸리는 거리다.... 
-----
https://blog.naver.com/j7447?Redirect=Log&logNo=220781580464
(대전 여행) 오월드 야간개장 르미에르 축제에 다녀왔습니다.
모처럼 시간을 맞추게 되서 하루 여행을 떠났어요. 어디 멀리가기엔 어렵고해서... 가까운 대전 오월드 야간 개장을 갔습니다. 오월드에 몇번 와보긴 했지만... 야간개장에 오는건 처음이예요! 들어가는 입구에서... 
-----
https://blog.naver.com/l9890ll9?Redirect=Log&logNo=220552249118
[충남여행/대전여행]

https://blog.naver.com/lovelyr929?Redirect=Log&logNo=220464836622
대전 여행 / 대전 동물원 * 오월드
대전여행의 제일 큰 목적은 '오월드'에 가는것이였다 첨으로 친구랑 둘이 여행간거였는데 정말 알차고 재밌게 놀았던가같다ㅋㅋㅋㅋ 외국인이 찍어준 사진은.. 호랑이 머리없이 우리만 덩그러니^^; 그래도... 
-----
https://blog.naver.com/sso5573?Redirect=Log&logNo=220609601481
대전여행  -오~월드 겨울은 정^^말  추워요
하늘과 별의 겨울방학 마지막 여행~~ 2월1일부터 개학하는 겨울방학을 앞두고 대전 오~월드에 왔더래요. 경남에서 대전까지 갈려고 하니 아침 일찍부터 서둘렀는 데~~ 중간점 휴게소에 잠깐 쉬는데 눈이... 
-----
https://blog.naver.com/yotnwjd?Redirect=Log&logNo=140187590477
[대전여행] 오월드 동물원/플라워랜드
대전하면 엑스포가 떠오르는데 거긴.. 초딩 수학여행으로 갔다왔으니 패스 ㅋㅋ 다른곳이 없을까해서 찾아보니 동물원/수목원 정도? 그래서 우린 '오월드' 너로 정했다. 대흥동에서 칼국수를 맛나게 먹고... 
-----
https://blog.naver.com/3dutk?Redirect=Log&logNo=20187361950
[김여사의 체험여행]오~~~~~월드-대전오월드.동물원.사파리.... 
주저마시고 ~대전의 오월드로 가세요~ http://www.oworld.kr/ 애버랜드.롯데월드와 비교하자면 너무나... 녹용이라도먹고 달리기 1등해보렵니다 ㅎㅎ 이번주말. 아이들과 갈때 없으시다구요? 떠나세요~오~월드로!!!
-----
https://blog.naver.com/meteorslove?Redirect=Log&logNo=100164800188
[여름휴가/가족여행/대전오월드] 대전유성리베라호텔, 대전오월드
더군다나 오늘은 완전 햇볕 노출인 &lt;대전오월드&gt; 어제 물놀이 탓에 많이 

In [9]:
naver_blog_crawling(data_dir, '오월드여행대전', 100, "date", fromdate, todate)    

키워드 오월드여행대전에 해당하는 포스팅 수 : 10484
키워드 오월드여행대전에 해당하는 블로그 실제 페이징 수 : 105
키워드 오월드여행대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 105
finish crawling !
crawl count : 535


In [ ]:
for word in complete_keyword:
    naver_blog_crawling(data_dir, word, 100, "date", 20190101, 20210501)    

키워드 오월드 여행 후기에 해당하는 포스팅 수 : 4
키워드 오월드 여행 후기에 해당하는 블로그 실제 페이징 수 : 1
키워드 오월드 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 오월드 여행 대전에 해당하는 포스팅 수 : 40
키워드 오월드 여행 대전에 해당하는 블로그 실제 페이징 수 : 1
키워드 오월드 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 오월드 여행 주말에에 해당하는 포스팅 수 : 2
키워드 오월드 여행 주말에에 해당하는 블로그 실제 페이징 수 : 1
키워드 오월드 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
finish crawling !
finish crawling !
키워드 오월드 관광 주말에에 해당하는 포스팅 수 : 7
키워드 오월드 관광 주말에에 해당하는 블로그 실제 페이징 수 : 1
키워드 오월드 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 뿌리공원 여행 후기에 해당하는 포스팅 수 : 2
키워드 뿌리공원 여행 후기에 해당하는 블로그 실제 페이징 수 : 1
키워드 뿌리공원 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 뿌리공원 여행 대전에 해당하는 포스팅 수 : 18
키워드 뿌리공원 여행 대전에 해당하는 블로그 실제 페이징 수 : 1
키워드 뿌리공원 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 뿌리공원 여행 주말에에 해당하는 포스팅 수 : 409
키워드 뿌리공원 여행 주말에에 해당하는 블로그 실제 페이징 수 : 5
키워드 뿌리공원 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 5
finish crawling !
키워드 뿌리공원 관광 후기에 해당하는 포스팅 수 : 2
키워드 뿌리공원 관광 후기에

In [10]:
complete_keyword[14]

'엑스포과학공원 여행 주말에'

In [50]:
for word in complete_keyword:
    naver_blog_crawling(data_dir, word, 100, "date")    

TypeError: naver_blog_crawling() missing 2 required positional arguments: 'fromdate' and 'todate'

In [52]:
for word in complete_keyword:
    naver_blog_crawling(data_dir, word, 100, "date", '20160101', '20210501')    

finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
finish crawling !
crawl count : 0
키워드 뿌리공원 여행 주말에에 해당하는 포스팅 수 : 11905
키워드 뿌리공원 여행 주말에에 해당하는 블로그 실제 페이징 수 : 11905
키워드 뿌리공원 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1000


InvalidURL: URL can't contain control characters. '/v1/search/blog?&query=뿌리공원 여행 주말에fromFalsetoFalse' (found at least ' ')

In [ ]:
for word in new_keyword:
    naver_blog_crawling(data_dir, word, 100, "date", 20190101, 20210501)    

In [28]:
for word in complete_keyword[72:78]:
    naver_blog_crawling(data_dir, word, 100, "date", 20190101, 20210501)    

키워드 성심당 여행 후기에 해당하는 포스팅 수 : 28
키워드 성심당 여행 후기에 해당하는 블로그 실제 페이징 수 : 1
키워드 성심당 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 성심당 여행 대전에 해당하는 포스팅 수 : 97
키워드 성심당 여행 대전에 해당하는 블로그 실제 페이징 수 : 1
키워드 성심당 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 성심당 여행 주말에에 해당하는 포스팅 수 : 7
키워드 성심당 여행 주말에에 해당하는 블로그 실제 페이징 수 : 1
키워드 성심당 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 성심당 관광 후기에 해당하는 포스팅 수 : 6
키워드 성심당 관광 후기에 해당하는 블로그 실제 페이징 수 : 1
키워드 성심당 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 성심당 관광 대전에 해당하는 포스팅 수 : 21
키워드 성심당 관광 대전에 해당하는 블로그 실제 페이징 수 : 1
키워드 성심당 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 성심당 관광 주말에에 해당하는 포스팅 수 : 1
키워드 성심당 관광 주말에에 해당하는 블로그 실제 페이징 수 : 1
키워드 성심당 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !


In [13]:
for word in complete_keyword[43:-1]:
    naver_blog_crawling(data_dir, word, 100, "sim")  

키워드 대전둘레산길 여행 대전에 해당하는 포스팅 수 : 38243
키워드 대전둘레산길 여행 대전에 해당하는 블로그 실제 페이징 수 : 383
키워드 대전둘레산길 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 383


HTTPError: HTTP Error 504: Gateway Time-out

In [14]:
for word in complete_keyword[44:-1]:
    naver_blog_crawling(data_dir, word, 100, "sim")  

키워드 대전둘레산길 여행 주말에에 해당하는 포스팅 수 : 3186
키워드 대전둘레산길 여행 주말에에 해당하는 블로그 실제 페이징 수 : 32
키워드 대전둘레산길 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 32
finish crawling !
키워드 대전둘레산길 관광 후기에 해당하는 포스팅 수 : 4573
키워드 대전둘레산길 관광 후기에 해당하는 블로그 실제 페이징 수 : 46
키워드 대전둘레산길 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 46
finish crawling !
키워드 대전둘레산길 관광 대전에 해당하는 포스팅 수 : 26899
키워드 대전둘레산길 관광 대전에 해당하는 블로그 실제 페이징 수 : 269
키워드 대전둘레산길 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 269


KeyboardInterrupt: 

In [ ]:
for word in complete_keyword[47:-1]:
    naver_blog_crawling(data_dir, word, 100, "sim")  

키워드 대전둘레산길 관광 주말에에 해당하는 포스팅 수 : 2232
키워드 대전둘레산길 관광 주말에에 해당하는 블로그 실제 페이징 수 : 23
키워드 대전둘레산길 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 23
finish crawling !
키워드 동춘당 여행 후기에 해당하는 포스팅 수 : 316
키워드 동춘당 여행 후기에 해당하는 블로그 실제 페이징 수 : 4
키워드 동춘당 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 4
finish crawling !
키워드 동춘당 여행 대전에 해당하는 포스팅 수 : 1208
키워드 동춘당 여행 대전에 해당하는 블로그 실제 페이징 수 : 13
키워드 동춘당 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 13
finish crawling !
키워드 동춘당 여행 주말에에 해당하는 포스팅 수 : 48
키워드 동춘당 여행 주말에에 해당하는 블로그 실제 페이징 수 : 1
키워드 동춘당 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 동춘당 관광 후기에 해당하는 포스팅 수 : 184
키워드 동춘당 관광 후기에 해당하는 블로그 실제 페이징 수 : 2
키워드 동춘당 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 2
finish crawling !
키워드 동춘당 관광 대전에 해당하는 포스팅 수 : 1048
키워드 동춘당 관광 대전에 해당하는 블로그 실제 페이징 수 : 11
키워드 동춘당 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 11
finish crawling !
키워드 동춘당 관광 주말에에 해당하는 포스팅 수 : 30
키워드 동춘당 관광 주말에에 해당하는 블로그 실제 페이징 수 : 1
키워드 동춘당 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 대전문화예술단지 여행 후기에 해당하는 포스팅 수 : 16125
키워드 대전문화예술단지 여행 후기에 해당하는 블

In [ ]:
for word in complete_keyword[60:-1]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 으능정이문화의거리 여행 후기에 해당하는 포스팅 수 : 55
키워드 으능정이문화의거리 여행 후기에 해당하는 블로그 실제 페이징 수 : 1
키워드 으능정이문화의거리 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 으능정이문화의거리 여행 대전에 해당하는 포스팅 수 : 503
키워드 으능정이문화의거리 여행 대전에 해당하는 블로그 실제 페이징 수 : 6
키워드 으능정이문화의거리 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 6
finish crawling !
키워드 으능정이문화의거리 여행 주말에에 해당하는 포스팅 수 : 169261
키워드 으능정이문화의거리 여행 주말에에 해당하는 블로그 실제 페이징 수 : 1693
키워드 으능정이문화의거리 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1000


In [ ]:
for word in complete_keyword[62:-1]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 으능정이문화의거리 여행 주말에에 해당하는 포스팅 수 : 169285
키워드 으능정이문화의거리 여행 주말에에 해당하는 블로그 실제 페이징 수 : 1693
키워드 으능정이문화의거리 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 1000


In [ ]:
for word in complete_keyword[63:-1]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 으능정이문화의거리 관광 후기에 해당하는 포스팅 수 : 23
키워드 으능정이문화의거리 관광 후기에 해당하는 블로그 실제 페이징 수 : 1
키워드 으능정이문화의거리 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 1
finish crawling !
키워드 으능정이문화의거리 관광 대전에 해당하는 포스팅 수 : 333
키워드 으능정이문화의거리 관광 대전에 해당하는 블로그 실제 페이징 수 : 4
키워드 으능정이문화의거리 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 4
finish crawling !
키워드 으능정이문화의거리 관광 주말에에 해당하는 포스팅 수 : 90562
키워드 으능정이문화의거리 관광 주말에에 해당하는 블로그 실제 페이징 수 : 906
키워드 으능정이문화의거리 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 906


In [ ]:
for word in complete_keyword[65:-1]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 으능정이문화의거리 관광 주말에에 해당하는 포스팅 수 : 90565
키워드 으능정이문화의거리 관광 주말에에 해당하는 블로그 실제 페이징 수 : 906
키워드 으능정이문화의거리 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 906


In [8]:
for word in complete_keyword[66:-1]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 유성온천 여행 후기에 해당하는 포스팅 수 : 1451
키워드 유성온천 여행 후기에 해당하는 블로그 실제 페이징 수 : 15
키워드 유성온천 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 15
finish crawling !
키워드 유성온천 여행 대전에 해당하는 포스팅 수 : 8336
키워드 유성온천 여행 대전에 해당하는 블로그 실제 페이징 수 : 84
키워드 유성온천 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 84
finish crawling !
키워드 유성온천 여행 주말에에 해당하는 포스팅 수 : 451
키워드 유성온천 여행 주말에에 해당하는 블로그 실제 페이징 수 : 5
키워드 유성온천 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 5
finish crawling !
키워드 유성온천 관광 후기에 해당하는 포스팅 수 : 579
키워드 유성온천 관광 후기에 해당하는 블로그 실제 페이징 수 : 6
키워드 유성온천 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 6
finish crawling !
키워드 유성온천 관광 대전에 해당하는 포스팅 수 : 6637
키워드 유성온천 관광 대전에 해당하는 블로그 실제 페이징 수 : 67
키워드 유성온천 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 67
finish crawling !
키워드 유성온천 관광 주말에에 해당하는 포스팅 수 : 182
키워드 유성온천 관광 주말에에 해당하는 블로그 실제 페이징 수 : 2
키워드 유성온천 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 2
finish crawling !
키워드 성심당 여행 후기에 해당하는 포스팅 수 : 3694
키워드 성심당 여행 후기에 해당하는 블로그 실제 페이징 수 : 37
키워드 성심당 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 37
finish crawling !
키워드 성심당 여행 대전에 해당하는 포스팅 수 : 24387
키워드 성심당 여행 대전에 해당하는 

In [6]:
for word in new_keyword:
    naver_blog_crawling(data_dir, word, 100, "sim")  

키워드 대전예술의전당 여행 후기에 해당하는 포스팅 수 : 985
키워드 대전예술의전당 여행 후기에 해당하는 블로그 실제 페이징 수 : 10
키워드 대전예술의전당 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 10
finish crawling !
키워드 대전예술의전당 여행 대전에 해당하는 포스팅 수 : 9421
키워드 대전예술의전당 여행 대전에 해당하는 블로그 실제 페이징 수 : 95
키워드 대전예술의전당 여행 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 95
finish crawling !
키워드 대전예술의전당 여행 주말에에 해당하는 포스팅 수 : 37640
키워드 대전예술의전당 여행 주말에에 해당하는 블로그 실제 페이징 수 : 377
키워드 대전예술의전당 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 377
finish crawling !
키워드 대전예술의전당 관광 후기에 해당하는 포스팅 수 : 318
키워드 대전예술의전당 관광 후기에 해당하는 블로그 실제 페이징 수 : 4
키워드 대전예술의전당 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 4
finish crawling !
키워드 대전예술의전당 관광 대전에 해당하는 포스팅 수 : 4345
키워드 대전예술의전당 관광 대전에 해당하는 블로그 실제 페이징 수 : 44
키워드 대전예술의전당 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 44
finish crawling !
키워드 대전예술의전당 관광 주말에에 해당하는 포스팅 수 : 21827
키워드 대전예술의전당 관광 주말에에 해당하는 블로그 실제 페이징 수 : 219
키워드 대전예술의전당 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 219
finish crawling !
키워드 대전시립미술관 여행 후기에 해당하는 포스팅 수 : 267
키워드 대전시립미술관 여행 후기에 해당하는 블로그 실제 페이징 수 : 3
키워드 대전시립미술관 여행 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 

URLError: <urlopen error [Errno -2] Name or service not known>

In [20]:
new_keyword[19], len(new_keyword)

('국립중앙과학관 여행 대전', 24)

In [ ]:
for word in new_keyword[20:]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 국립중앙과학관 여행 주말에에 해당하는 포스팅 수 : 39917
키워드 국립중앙과학관 여행 주말에에 해당하는 블로그 실제 페이징 수 : 400
키워드 국립중앙과학관 여행 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 400


In [10]:
for word in new_keyword[21:]:
    naver_blog_crawling(data_dir, word, 100, "date")  

키워드 국립중앙과학관 관광 후기에 해당하는 포스팅 수 : 161
키워드 국립중앙과학관 관광 후기에 해당하는 블로그 실제 페이징 수 : 2
키워드 국립중앙과학관 관광 후기에 해당하는 블로그 처리할 수 있는 페이징 수 : 2
finish crawling !
키워드 국립중앙과학관 관광 대전에 해당하는 포스팅 수 : 1673
키워드 국립중앙과학관 관광 대전에 해당하는 블로그 실제 페이징 수 : 17
키워드 국립중앙과학관 관광 대전에 해당하는 블로그 처리할 수 있는 페이징 수 : 17
finish crawling !
키워드 국립중앙과학관 관광 주말에에 해당하는 포스팅 수 : 27537
키워드 국립중앙과학관 관광 주말에에 해당하는 블로그 실제 페이징 수 : 276
키워드 국립중앙과학관 관광 주말에에 해당하는 블로그 처리할 수 있는 페이징 수 : 276
finish crawling !
